## LangChain Expression Language LCEL
- 다양한 구성 요소를 단일 체인으로 결합할 때 사용
- LCEL이 무엇인지, 어떻게 작동하는지, LCEL 체인, 파이프 및 Runnables의 필수 요소

#### pinecone에선 Claude 모델사용
- gpt 사용하고 싶다면 
model = ChatOpenAI(
    model="gpt-3.5-turbo" 
    max_tokens=512,
    openai_api_key=OPENAI_API_KEY
)
- claude 14일 동안만 무료로 사용가능

In [ ]:
ANTHROPIC_API_KEY = ''

In [2]:
from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

ANTHROPIC_API_KEY = ANTHROPIC_API_KEY

prompt = ChatPromptTemplate.from_template(
    "Give me small report about {topic}"
)

In [3]:
model = ChatAnthropic(
    model="claude-2.1",
    max_tokens_to_sample=512,
    anthropic_api_key=ANTHROPIC_API_KEY
)  # swap Anthropic for OpenAI with `ChatOpenAI` and `openai_api_key`
output_parser = StrOutputParser()

In [5]:
from langchain.chains import LLMChain

chain = LLMChain(
    prompt=prompt,
    llm=model,
    output_parser=output_parser
)

# and run
out = chain.run(topic="Artificial Intelligence")
print(out)

 Here is a brief report on artificial intelligence:

Artificial intelligence (AI) refers to computer systems that are designed to perform tasks and functions that normally require human intelligence. AI has seen tremendous advancements over the past decade due to faster computing power and the availability of vast amounts of data. Some current capabilities of AI systems include:

- Image and speech recognition - AI algorithms have become good at recognizing images and speech. For example, face recognition allows smartphones to unlock by scanning a user's face.

- Natural language processing - AI enables machines to read, understand, and derive meaning from human languages. This powers chatbots and virtual assistants like Siri and Alexa.

- Expert systems - These provide recommendations and analysis based on rules and data sets. For example, IBM's Watson diagnoses health issues by analyzing medical history, symptoms, test results, and other data.

- Self-driving cars - AI algorithms all

- 위의 코드는 지금까지 사용했던 코드
- 아래 코드가 lcel 사용
- 왼쪽에서 출력을 받아 오른쪽으로 입력 

In [6]:
lcel_chain = prompt | model | output_parser

# and run
out = lcel_chain.invoke({"topic": "Artificial Intelligence"})
print(out)

 Here is a brief report on artificial intelligence:

Introduction:
Artificial intelligence (AI) refers to computer systems or machines that are designed to perform human-like cognitive functions such as learning, understanding, reasoning, and self-correction. AI science aims to create intelligent machines that can think and act rationally. Significant progress over the past few decades has led to AI systems matching or surpassing human capabilities in several areas.

Current Applications of AI: 
Some current applications where AI programs have shown impressive results include:
- Computer vision - Image recognition, facial recognition, self-driving cars
- Natural language processing - Machine translation, conversational agents
- Game playing - Chess, Go, video games
- Decision-making and predictions - Stock trading, medical diagnosis, weather forecasting
- Robotics and smart machines - Assembly line robots, delivery drones, smart appliances

Future Possibilities:
While current AI progra

#### 보다 정확하게 이해하기 위해 custom pipe operator

- object approach
chain = a.__or__(b)
chain("some input")
- pipe approach
chain = a | b
chain("some input")
- a를 b에

-  __ or __ : Runnable 인스턴스를 연결

In [9]:
class Runnable:
    def __init__(self, func):
        self.func = func

    def __or__(self, other):
        def chained_func(*args, **kwargs):
            # the other func consumes the result of this func
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [10]:
def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2


add_five = Runnable(add_five)
multiply_by_two = Runnable(multiply_by_two)

# run them using the object approach
chain = add_five.__or__(multiply_by_two)
chain(3)  

16

- add_five에 먼저 들어가고, 이 출력값이 mul 

In [12]:
chain = add_five | multiply_by_two

chain(3)

16

## Deep Dive

## Runnables
- 작업 유연성 극대화 
- 벡터 저장소 구성요소 초기화 

In [13]:
import os
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [14]:
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_community.embeddings import OpenAIEmbeddings

embed = OpenAIEmbeddings(model = 'text-embedding-ada-002',
                         openai_api_key = OPENAI_API_KEY
                        )

C:\Users\sy\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [15]:
vecstore_a = DocArrayInMemorySearch.from_texts(
    ["half the info will be here", "James' birthday is the 7th December"],
    embedding=embed
)
vecstore_b = DocArrayInMemorySearch.from_texts(
    ["and half here", "James was born in 1994"],
    embedding=embed
)

C:\Users\sy\anaconda3\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [16]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

prompt_str = """Answer the question below using the context:

Context: {context}

Question: {question}

Answer: """

In [17]:
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {"context": retriever_a, "question": RunnablePassthrough()}
)

chain = retrieval | prompt | model | output_parser

-  RunnableParallel과 RunnablePassthrough라는 두 개의 새로운 개체를 사용합니다. 
- RunnableParallel 개체를 사용하면 여러 개의 값과 연산을 정의하고 이들을 모두 병렬로 실행할 수 있음. 
- 여기서는 체인(아래)에 대한 입력을 사용하여 retriever_a를 호출한 다음 "context" 매개 변수를 통해 결과를 체인의 다음 구성 요소로 전달

#### RunnableParallel
- 여러 작업을 병렬로 실행함. 해당 작업이 동시에 실행
- 내부의 작업은 동일한 입력을 받고 독립적으로 사용하거나 나중에 결합할 수 있는 다양한 출력을 생성

#### RunnablePassthrough
- 입력을 수정하지 않고 체인의 현재 단계를 통해 직접 전달
- 원래 입력 또는 입력/출력의 일부를 전달하여 다음 단계에서 사용가능 

In [21]:
out = chain.invoke("when was James born?")
print(out)

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [0.00463924... -0.013804269162738882]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [0.00463924... -0.013804269162738882]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing

In [22]:
prompt_str = """Answer the question below using the context:

Context:
{context_a}
{context_b}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b,
        "question": RunnablePassthrough()
    }
)

chain = retrieval | prompt | model | output_parser

In [23]:
out = chain.invoke("when was James born?")
print(out)

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [0.00463924... -0.013804269162738882]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [0.00463924... -0.013804269162738882]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing

In [24]:
out = chain.invoke("what date exactly was James born?")
print(out)

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [-0.0087266..., -0.01529460373632836]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [-0.0087266..., -0.01529460373632836]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing

## Runnable Lambdas

In [25]:
from langchain_core.runnables import RunnableLambda

def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2

# wrap the functions with RunnableLambda
add_five = RunnableLambda(add_five)
multiply_by_two = RunnableLambda(multiply_by_two)

In [26]:
chain = add_five | multiply_by_two

In [27]:
chain.invoke(3)

16

In [28]:
chain.invoke(5)

20

- 사용자 custom func chain 이용해보기 

In [29]:
prompt_str = "Tell me an short fact about {topic}"
prompt = ChatPromptTemplate.from_template(prompt_str)

chain = prompt | model | output_parser

In [30]:
chain.invoke({"topic": "Artificial Intelligence"})

" Here's a short fact about artificial intelligence:\n\nAI systems can analyze data and identify patterns that would be impossible or very difficult for humans to spot on their own. This ability to find insights in data is one of the key strengths of AI and enables many practical applications."

In [31]:
chain.invoke({"topic": "Artificial Intelligence"})

" Here's a short fact about artificial intelligence:\n\nAI systems can analyze huge amounts of data faster than humans ever could. Using machine learning algorithms, AI systems can detect patterns and make predictions based on large datasets in ways that would be practically impossible for people. This ability to quickly process high volumes of data and find insights is one of the key strengths of artificial intelligence."

- 텍스트의 특정 부분을 추출
- \n\n이 포함되어 있으면 텍스트 split
- [1:] : 항목을 \n으로 구분된 단일 문자열로 다시 결합
- 첫번째부분은 제거, 나머지 유지하도록 함. here's ~ 부분 제거
- chain 생성

In [32]:
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x
    
get_fact = RunnableLambda(extract_fact)

chain = prompt | model | output_parser | get_fact

In [33]:
chain.invoke({"topic": "Artificial Intelligence"})

'AI systems can analyze data and find patterns that humans may miss. For example, AI algorithms are very good at detecting credit card fraud by noticing small anomalies in spending patterns that indicate potentially fraudulent activity.'

- runnable 이해 더 필요